In [11]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import gradio as gr 

Force Dark Mode - for the sake of my eyes

In [12]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [13]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyBR


In [14]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

google.generativeai.configure()

Website Class

In [15]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [26]:
# Some of these prompts could use some work as they don't seem to exemplify some of the themes I was hoping they would achieve. The gothic_system_message's brochure is not near Edgar Allan Poe enough for my liking.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

humorous_system_message = "You are a witty and entertaining assistant that analyzes the contents of a \
company website landing page and crafts a fun, engaging brochure for prospective customers, investors, \
and recruits. Your responses should be humorous, lighthearted, and sprinkled with clever wordplay. \
Respond in markdown."

serious_system_message = "You are a highly professional and detail-oriented assistant that \
analyzes the contents of a company website landing page and produces a formal, well-structured brochure. \
Your tone is direct, no-nonsense, and suitable for corporate and executive-level audiences. \
Respond in markdown."

gothic_system_message = "You are a dark and mysterious assistant that analyzes the contents of a \
company website landing page and weaves an atmospheric, gothic-style brochure. Your tone is rich, \
melancholic, and poetic, invoking imagery of shadowy corridors, timeless grandeur, and the eerie \
passage of time. Respond in markdown."

scientific_system_message = "You are a precise and analytical assistant that examines the contents of a \
company website landing page and generates a data-driven, research-backed brochure. Your language is \
technical, objective, and rooted in logical reasoning, making the information accessible yet informative \
for scientifically-minded customers, investors, and recruits. Respond in markdown."


Modifying Stream function to accept multiple tones

In [27]:
def stream_gpt(prompt, tone="default"):
    # Dictionary mapping tones to predefined system messages
    system_messages = {
        "default": system_message,
        "humorous": humorous_system_message,
        "serious": serious_system_message,
        "gothic": gothic_system_message,
        "scientific": scientific_system_message
    }

    # Select the system message based on the chosen tone (default if not found)
    system_message_selected = system_messages.get(tone, system_message)

    messages = [
        {"role": "system", "content": system_message_selected},
        {"role": "user", "content": prompt}
    ]

    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


Stream GPT Response

In [ ]:
# def stream_gpt(prompt):
#     messages = [
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": prompt}
#       ]
#     stream = openai.chat.completions.create(
#         model='gpt-4o-mini',
#         messages=messages,
#         stream=True
#     )
#     result = ""
#     for chunk in stream:
#         result += chunk.choices[0].delta.content or ""
#         yield result

In [28]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    # elif model=="Claude":
    #     result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [29]:
def stream_brochure(company_name, url, model, tone):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    # elif model=="Claude":
    #     result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [32]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model"),
        gr.Dropdown(["Default", "Humorous", "Serious", "Gothic", "Scientific"], label="Select Tone")
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never",
    js=force_dark_mode
)

view.launch()


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


In [33]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude"], label="Select model"),
        gr.Dropdown(["Default", "Humorous", "Serious", "Gothic", "Scientific"], label="Select Tone")
    ],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never",
    js=force_dark_mode
)

view.launch()


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.
